In [1]:
from instruct_tri2tri.tsr.system import InstructTri2Tri, TSR
from torch.utils.data import Dataset
import torch
import json
from tqdm import tqdm

class InstructTri2TriDataset(Dataset):
    def __init__(self,
                 data_path):
        super().__init__()
        datas = json.load(open(data_path))[:200]
        self.datas = datas

    def __len__(self):
        return len(self.datas)
    
    def __getitem__(self, index):
        data = self.datas[index]
        image_name = data['image']
        instruct_image_name = data['instruct_image']
        image = Image.open(f'data/{image_name}')
        instruct_image = Image.open(f'data/{instruct_image_name}')
        # image_name = image_name.split('/')[-1]
        instruct = data['instruct']
        return image, instruct_image, instruct


/root/data/anaconda3/envs/mobilevlm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = InstructTri2Tri.from_pretrained(
    'instruct_tri2tri/tsr/instruct_tri2tri_config',
    config_name="config.yaml",
    weight_name="model.ckpt",
)
dataset = InstructTri2TriDataset('data/objaverse/cap3d_automated_objaverse_highquality_instruct_550k.json')

In [3]:
model.requires_grad_(False)
model.image_tokenizer.requires_grad_(False)
model.tokenizer.requires_grad_(False)
model.text_encoder.requires_grad_(False)
model.backbone.requires_grad_(False)
model.post_processor.requires_grad_(False)
model.decoder.requires_grad_(False)
model.renderer.requires_grad_(False)
model.instruction_converter.requires_grad_(True)
model.cuda(2)

InstructTri2Tri(
  (image_tokenizer): DINOSingleImageTokenizer(
    (model): ViTModel(
      (embeddings): ViTEmbeddings(
        (patch_embeddings): ViTPatchEmbeddings(
          (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): ViTEncoder(
        (layer): ModuleList(
          (0-11): 12 x ViTLayer(
            (attention): ViTAttention(
              (attention): ViTSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): ViTSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
 

In [4]:
optimizer = torch.optim.AdamW(model.instruction_converter.parameters(), lr=2e-5)
loss_fn = torch.nn.MSELoss()
device = 'cuda:2'

In [5]:
from PIL import Image
image, instruct_image, instruct = dataset[0]
with torch.no_grad():
    target_tokens = model.forward_tsr([instruct_image], device, False)
torch.cuda.empty_cache()
pred_tokens = model([image], [instruct], device, False)
dim = target_tokens.shape[2]
loss = loss_fn(pred_tokens.reshape(-1, dim), target_tokens.reshape(-1, dim))

In [6]:
image, instruct_image, instruct = dataset[1]
with torch.no_grad():
    target_tokens = model.forward_tsr([instruct_image], device, False)
torch.cuda.empty_cache()
pred_tokens = model([image], [instruct], device, False)
dim = target_tokens.shape[2]
loss = loss_fn(pred_tokens.reshape(-1, dim), target_tokens.reshape(-1, dim))

OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB (GPU 2; 23.70 GiB total capacity; 2.54 GiB already allocated; 37.56 MiB free; 2.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
from PIL import Image
for image, instruct_image, instruct in tqdm(dataset):
    with torch.no_grad():
        target_tokens = model.forward_tsr([instruct_image], device, False)
    torch.cuda.empty_cache()
    pred_tokens = model([image], [instruct], device, False)
    dim = target_tokens.shape[2]
    loss = loss_fn(pred_tokens.reshape(-1, dim), target_tokens.reshape(-1, dim))
    # loss.backward()
    # optimizer.step()
    # optimizer.zero_grad()

  0%|          | 1/200 [00:06<20:02,  6.04s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB (GPU 2; 23.70 GiB total capacity; 2.54 GiB already allocated; 37.56 MiB free; 2.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from transformers import CLIPTextModelWithProjection, AutoTokenizer

clip = CLIPTextModelWithProjection.from_pretrained('ckpts/clip-vit-large-patch14').cuda()
tokenizer = AutoTokenizer.from_pretrained('ckpts/clip-vit-large-patch14')

In [ ]:
inputs = tokenizer(['asdasdasdasd', 'sadasdasd'], padding=True, return_tensors='pt')
for key, value in inputs.items():
    inputs[key] = value.cuda()
out = clip(**inputs)

In [ ]:
out.keys()

odict_keys(['text_embeds', 'last_hidden_state'])

In [ ]:
out.text_embeds.shape

torch.Size([2, 768])